In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(valid_pack_raw)
predict_pack_processed = preprocessor.transform(predict_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9820.48it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5371.84it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2897487.29it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10390.14it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5278.59it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 62883.80it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9913.12it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:0

In [4]:
model = mz.models.DRMMTKS()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['mask_value'] = -1
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()
print(model.params)

model_class                   <class 'matchzoo.models.drmmtks.DRMMTKS'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adadelta
with_embedding                True
embedding_input_dim           16674
embedding_output_dim          100
embedding_trainable           True
with_multi_layer_perceptron   True
mlp_num_units                 5
mlp_num_layers                1
mlp_num_fan_out               1
mlp_activation_func           relu
mask_value                    -1
top_k                         20


In [5]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 10, 100)      0           embedding[0][0]                  
          

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
model.load_embedding_matrix(embedding_matrix)

In [8]:
pred_x, pred_y = predict_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [9]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
len(train_generator)

102

In [10]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=4, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 2s 21ms/step - loss: 0.9896
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.339367254494009 - normalized_discounted_cumulative_gain@5(0.0): 0.4207738396914389 - mean_average_precision(0.0): 0.3927174637148217
Epoch 2/30
102/102 [==============================] - 4s 44ms/step - loss: 0.9077
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.43529739522463656 - normalized_discounted_cumulative_gain@5(0.0): 0.49828231750958146 - mean_average_precision(0.0): 0.4777063279813898
Epoch 3/30
102/102 [==============================] - 5s 46ms/step - loss: 0.8827
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4081034257047157 - normalized_discounted_cumulative_gain@5(0.0): 0.46788166466955333 - mean_average_precision(0.0): 0.4396750654434257
Epoch 4/30
102/102 [==============================] - 5s 47ms/step - loss: 0.8252
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4208558564583984 - normalize

In [11]:
append_params_to_readme(model)